## Customer Segmentation RFM

## **Content:**
"Thistransnational data seth contains all the transactions occurring between 01/12/2010 and 09/12/2011 for a UK-based and registered non-store online retail .The company mainly sells unique all-occasion gifts. Many customers of the company are wholesalers For more: https://www.kaggle.com/datasets/carrie1/ecommerce-data"

Customer Segmentation using RFM (Recency, Frequency, Monetary) is a method used to identify and categorize customers based on their purchasing behavior, allowing businesses to tailor marketing strategies effectively. Recency measures how recently a customer made a purchase, Frequency evaluates how often they purchase, and Monetary assesses the total spending of a customer. By analyzing these three dimensions, businesses can segment customers into distinct groups, such as loyal customers, big spenders, or at-risk customers. This segmentation helps in understanding the value of different customer groups, predicting future behaviors, and creating targeted marketing campaigns to enhance customer retention and maximize revenue.